In [ ]:
import re
import pandas as pd
from datetime import datetime

def is_insert(line):
    return 'INSERT INTO' in line or False

def is_create(line):
    return 'CREATE TABLE' in line or False

def is_finish(line):
    return ';' in line or False

def is_primary(line):
    return 'PRIMARY KEY' in line or False

def get_values(line):
    return line.partition(' VALUES ')[2]

def get_table_name_insert(line):
    match = re.search('INSERT INTO `([0-9_a-zA-Z]+)`', line)
    if match:
        return match.group(1)
    else:
        print(line)

def get_name_categorie_type(line):
    match = re.search('int\([0-9]+\)', line)
    if match:
        return "int"
    else:
        match = re.search('char', line)
        if match:
            return match.group(0)
        else:
            match = re.search('date', line)
            if match:
                return match.group(0)
            else:
                match = re.search('time', line)
                if match:
                    return match.group(0)
                else:
                    match = re.search('text', line)
                    if match:
                        return match.group(0)
                    else:
                        print(line)
              
        
def get_table_name_create(line):
    match = re.search('CREATE TABLE IF NOT EXISTS `([0-9_a-zA-Z]+)`', line)
    if match:
        return match.group(1)
    else:
        print(line)
        
def get_name_categorie(line):
    match = re.search('`([0-9_a-zA-Z]+)`', line)
    if match:
        return match.group(1)
    else:
        print(line)
      
    
def get_columns(line):
    match = re.search('INSERT INTO `.*` \(([^\)]+)\)', line)
    if match:
        return list(map(lambda x: x.replace('`', '').strip(), match.group(1).split(',')))

def parse_line(line):
    if line[0] == "(": 
        line = line[1:-2]
        line = line.split(", ")
    return line
    
def sqltopanda(filename):
    d = {}
    type_categorie = {}
    values = []
    table_name = []
    columns = []
    insert = False
    create = False
    reader = open(filename, 'r', encoding='utf-8')
    contenu = reader.read()
    lines = contenu.split("\n")
    j =0
    
    for line in lines:
        if is_insert(line):
            insert = True
        elif is_create(line):
            table = get_table_name_create(line)
            d[get_table_name_create(line)] = pd.DataFrame()
            type_categorie[get_table_name_create(line)] = []
            create = True
        elif is_finish(line) and create:
            create = False
        if create and not is_create(line) and not is_primary(line):
            d[table][get_name_categorie(line)] = ""
            type_categorie[table].append(get_name_categorie_type(line))
        if (insert and not is_insert(line)) or (is_finish(line) and insert):
            dictio = {}
            values = parse_line(line)         
            for i in range(len(values)):
                if values[i] == "NULL":
                    d[table][list(d[table].columns)[i]] = values[i]
                elif type_categorie[table][i] == "int":
                    values[i] = int(values[i])
                elif type_categorie[table][i] == "char":
                    values[i] = str(values[i])
                elif type_categorie[table][i] == "date":
                    values[i] = datetime.strptime(values[i], "'%Y-%m-%d'").date()
                elif type_categorie[table][i] == "time":
                    values[i] = datetime.strptime(values[i], "'%H:%M:%S'").time()
                else:
                    print(type_categorie[table][i], values[i], type(values[i]))
                dictio[d[table].columns[i]] = values[i]
            d[table] = d[table].append(dictio, ignore_index=True)
            if is_finish(line) and insert:
                insert = False
    return d, type_categorie

# current date and time
#deb = datetime.now()
sql, type_categorie = sqltopanda("traceforum.sql")
#fin = datetime.now()
#print("déubt =", sql)
#print("fin =", type_categorie)

#sql.to_csv('data.csv')


In [ ]:
#sql=pd.read_csv('data.csv')

In [ ]:
#---------------------------------P A R T I E      D Y L A N       P A S     F I N I -----------------------------------------


def nbActions(Action, Utilisateur):
    nbMessage = sql['transition'][(sql['transition']['Titre'] == Action) & (sql['transition']['Utilisateur'] == Utilisateur)].count()['IDTran']


In [ ]:
Action = ["'Répondre à un message'", "'Poster un nouveau message'"]


for utilisateur in sql['transition']["Utilisateur"].unique().tolist():
    for act in Action:
        nbActions(act, utilisateur)


In [5]:
def postsUtilisateur(utilisateur):
    tab = []
    attribut = sql['transition']['Attribut'][(sql['transition']['Titre'] == "'Poster un nouveau message'") & (sql['transition']['Utilisateur'] == utilisateur)]
    for attr in attribut:
        tab.append(str(attr).split(",")[1].split("\n")[0].split("=")[1][:-1])
    return(tab)
    
    
def getReponsesPost(idParent):
    utilisateur = sql['transition']['Utilisateur'][(sql['transition']['Attribut'].str.contains('IDParent='+idParent)) & (sql['transition']['Titre'] == "'Répondre à un message'")]
    attribut = sql['transition']['Attribut'][(sql['transition']['Attribut'].str.contains('IDParent='+idParent))]
    if(attribut.empty):
        return ("", "")
    idMessage = str(attribut).split(",")[1].split("\n")[0].split("=")[1]
    return (utilisateur, idMessage)


In [6]:
#pour chaque utilisateur
import numpy as np
deb = datetime.now()
utilisateurs = sql['transition']["Utilisateur"].unique().tolist()
for l in range(len(utilisateurs)):
    print("\n Pour l'utilisateur :" + utilisateurs[l] +"\n" )
    #on initialise la matrice de résultat
    a = np.zeros(shape=(1,len(utilisateurs)))
    tableauResultat = pd.DataFrame(a, index = ['l1'], columns = utilisateurs)
    #on recupere les posts qu'il a créé
    posts = postsUtilisateur(utilisateurs[l])
    
    #pour chaque post qu'il a créé
    for i in range(len(posts)):
        tableauUsers = []
        user, idMessage = getReponsesPost(posts[i])
        #tant qu'il y a des messages on déroule le fil de la discussion
        while idMessage != "" :  
            #on récupere l'identifiant du message et l'utilisateur qui a posté le message
            tableauUsers.append(user.values[0])
            user, idMessage = getReponsesPost(idMessage)
        tableauUsers = list(set(tableauUsers))
       
        # on ajoute 1 dans la matrice si l'utilisateur a répondu au post
        for j in range(len(utilisateurs)):
            for k in range(len(tableauUsers)):
                if tableauUsers[k] == utilisateurs[j]:
                    tableauResultat.at['l1', utilisateurs[j]] = (1 + tableauResultat.at['l1', utilisateurs[j]])
   
    # on calcule la matrice en pourcentage
    if (len(posts) != 0) :
        tableauResultat = (tableauResultat * 100) / len(posts)
    print(tableauResultat)
fin = datetime.now()
print(deb)
print(fin)


 Pour l'utilisateur :'tdelille'

    'tdelille'    'mmay'  'ddaniaupotter'  'gachort'  'sebastien'  'admin'  \
l1   52.941176  5.882353        48.529412  29.411765          0.0      0.0   

    'madeth'  'cdejean'  'fdarriet'  'mwollenburger'  ...  'jvasseur'  \
l1       0.0        0.0         0.0        23.529412  ...         0.0   

    'mdanet'  'psalam'  'guest'  'cevanen'  'cfroger'  'adebeuckelaere'  \
l1       0.0       0.0      0.0        0.0        0.0               0.0   

    'shernu'  'xjmechain'  'ntricoit'  
l1       0.0          0.0         0.0  

[1 rows x 23 columns]

 Pour l'utilisateur :'mmay'

    'tdelille'     'mmay'  'ddaniaupotter'  'gachort'  'sebastien'  'admin'  \
l1   28.571429  42.857143        14.285714  28.571429          0.0      0.0   

    'madeth'  'cdejean'  'fdarriet'  'mwollenburger'  ...  'jvasseur'  \
l1       0.0        0.0         0.0        14.285714  ...         0.0   

    'mdanet'  'psalam'  'guest'  'cevanen'  'cfroger'  'adebeuckelaere' 

    'tdelille'  'mmay'  'ddaniaupotter'  'gachort'  'sebastien'  'admin'  \
l1       3.125    6.25              0.0        0.0          0.0      0.0   

    'madeth'  'cdejean'  'fdarriet'  'mwollenburger'  ...  'jvasseur'  \
l1       0.0      9.375      34.375              0.0  ...      21.875   

    'mdanet'  'psalam'  'guest'  'cevanen'  'cfroger'  'adebeuckelaere'  \
l1    28.125       0.0      0.0        0.0        0.0               0.0   

    'shernu'  'xjmechain'  'ntricoit'  
l1       0.0          0.0         0.0  

[1 rows x 23 columns]

 Pour l'utilisateur :'psalam'

    'tdelille'  'mmay'  'ddaniaupotter'  'gachort'  'sebastien'  'admin'  \
l1         0.0     0.0              0.0        0.0          0.0      0.0   

    'madeth'  'cdejean'  'fdarriet'  'mwollenburger'  ...  'jvasseur'  \
l1       0.0        0.0         0.0              0.0  ...         0.0   

    'mdanet'  'psalam'  'guest'  'cevanen'  'cfroger'  'adebeuckelaere'  \
l1       0.0      50.0      0.0      10

In [7]:
def getReponses(action, utilisateur):
     return(sql['transition'][(sql['transition']['Titre'] == action) & (sql['transition']['Utilisateur'] == utilisateur)])

def getMessage(idMsg):
     return(sql['transition'][((sql['transition']['Titre'].str.contains('Répondre à un message'))
                              |sql['transition']['Titre'].str.contains('Poster un nouveau message'))
                             & sql['transition']['Attribut'].str.contains('IDMsg='+idMsg)])

from datetime import datetime, date

def getDifferenceMessages(dateMessage, timeMessage, dateParent, timeParent):
    return(datetime.combine(dateMessage, timeMessage) - datetime.combine(dateParent, timeParent)).total_seconds()


 



def temps_moyen(seconds):
    intervals = (
    ('semaines', 604800),  # 60 * 60 * 24 * 7
    ('jours', 86400),    # 60 * 60 * 24
    ('heures', 3600),    # 60 * 60
    ('minutes', 60),
    ('secondes', 1),
    )
    result = []
    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(value, name))
    return ', '.join(result[:5])

In [8]:

for utilisateur in sql['transition']["Utilisateur"].unique().tolist():
    reponses = getReponses("'Répondre à un message'", utilisateur)
    tabDuree = 0
    i = 0
    for reponse in reponses.values:
        i = i+1
        dateMessage = reponse[4]
        timeMessage = reponse[5]
        timeParent = reponse[2][5]
        idParent = reponse[3].split(",")[2].split("=")[1][:-1]
        msgParent = getMessage(idParent)
        if(len(msgParent.values) != 0):
            dateParent = msgParent.values[0][4]
            timeParent = msgParent.values[0][5]
            tabDuree = tabDuree + getDifferenceMessages(dateMessage, timeMessage, dateParent, timeParent)
     
    if(i != 0):
        print(temps_moyen(tabDuree/i))

4.0 heures, 29.0 minutes, 57.0 secondes
10.0 heures, 24.0 minutes, 11.0 secondes
5.0 heures, 43.0 minutes, 25.0 secondes
3.0 heures, 51.0 minutes, 7.0 secondes
12.0 heures, 13.0 minutes, 29.0 secondes
2.0 jours, 1.0 heure, 4.0 minutes, 30.0 secondes
14.0 heures, 49.0 minutes, 39.0 secondes
7.0 heures, 27.0 minutes, 23.0 secondes
1.0 jour, 12.0 heures, 15.0 minutes, 10.0 secondes

8.0 heures, 1.0 minute, 7.0 secondes
11.0 heures, 14.0 minutes, 23.0 secondes
9.0 heures, 58.0 minutes, 57.0 secondes
2.0 jours, 4.0 heures, 58.0 minutes, 58.0 secondes
1.0 jour, 7.0 heures, 25.0 minutes, 49.0 secondes
1.0 jour, 9.0 heures, 23.0 minutes, 22.0 secondes
3.0 jours, 19.0 heures, 30.0 minutes, 48.0 secondes
47.0 secondes
5.0 heures, 51.0 minutes, 22.0 secondes


In [ ]:
def actionDate(Action, Utilisateur, Date):
    return sql['transition'][(sql['transition']['Titre'] == Action) & (sql['transition']['Utilisateur'] == Utilisateur) & (sql['transition']['Date'] == datetime.strptime(Date, "'%Y-%m-%d'").date())].count()['Date']

print(actionDate("'Connexion'", "'tdelille'","'2009-02-12'"))
print(actionDate("'Afficher le contenu d''un message'", "'mmay'","'2009-02-13'"))
print(actionDate("'Répondre à un message'", "'mmay'","'2009-02-13'"))

In [ ]:
def action2Date(Action, Utilisateur, Date1,Date2):
    List=[]
    
    for d in sql['transition']['Date'].tolist():
        if ((d >= datetime.strptime(Date1, "'%Y-%m-%d'").date()) and (d <= datetime.strptime(Date2, "'%Y-%m-%d'").date())):
            List.append(actionDate(Action, Utilisateur, str(d)))

    return List.sum()

print(action2Date("'Connexion'", "'tdelille'","'2009-02-12'", "'2009-02-13'"))
